# 0 - Install & Import packages

In [3]:
!pip install nnetsauce numpy scipy scikit-learn pandas tqdm joblib category_encoders GPopt BCN xgboost tabpfn

  Using cached tabpfn-0.1.10-py3-none-any.whl.metadata (5.6 kB)
  Using cached ucimlrepo-0.0.7-py3-none-any.whl.metadata (5.5 kB)
Using cached tabpfn-0.1.10-py3-none-any.whl (156 kB)
Using cached ucimlrepo-0.0.7-py3-none-any.whl (8.0 kB)


In [3]:
import BCN as bcn
import category_encoders as ce
import GPopt as gp
import joblib
import nnetsauce as ns
import numpy as np
import pandas as pd
import sklearn.datasets
import warnings
import xgboost as xgb

from sklearn.datasets import load_wine, load_iris, load_breast_cancer, make_classification
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.metrics import make_scorer, balanced_accuracy_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from tabpfn import TabPFNClassifier
from time import time
from functools import lru_cache

pd.options.display.float_format = '{:,.4f}'.format

# 1 - utility funcs and global variables

In [1]:
DATASETS = ["iris", "wine",
            "breast_cancer", "make"]

NROWS = 1000
NCOLS = 10

In [5]:
def replace_nan_with_median(arr):
    # Calculate the median of each column ignoring NaN values
    median_vals = np.nanmedian(arr, axis=0)

    # Iterate over each column index and replace NaN with the corresponding median value
    for col_idx in range(arr.shape[1]):
        col_values = arr[:, col_idx]
        nan_indices = np.isnan(col_values)
        if np.any(nan_indices):
            col_values[nan_indices] = median_vals[col_idx]

    return arr

def select_NROWS_NCOLS(X, y):
    print(f"X.shape (initial): {X.shape}")
    print(f"y.shape (initial): {y.shape}")
    print("Encoding features and response...")
    le = LabelEncoder()
    encoder = ce.HashingEncoder(return_df=False)
    X = np.asarray(encoder.fit_transform(X, y)).astype(np.float32)
    y = np.asarray(le.fit_transform(y)).astype(np.uint8)
    print("...Done.")
    print(f"Finding top {NCOLS} features if necessary...")
    if X.shape[1] > NCOLS:
        rf = RandomForestClassifier(n_estimators=50, random_state=42)
        rf.fit(X, y)
        indices = np.argsort(rf.feature_importances_)[::-1]
        top_cols = indices[:NCOLS]
        print(f"  top {NCOLS} indices: {top_cols}")
        X = X[:,top_cols]
        print(f"  X reduced shape: {X.shape}")
    print("...Done.")
    if X.shape[0] > NROWS:
      print(f"Subsampling to {NROWS} if necessary...")
      start = time()
      sub = ns.SubSampler(y=y.ravel().astype(np.uint8),
                          n_samples=NROWS, seed=123, n_jobs=-1)
      idx_rows  = sub.subsample().ravel()
      print(f"... Elapsed time for subsampling: {time() - start}")
      print("Number of rows in the subsample: ", len(idx_rows))
      print("Rows in the subsample: ", idx_rows)
      return_X = replace_nan_with_median(X[idx_rows,:])
      return_y = y[idx_rows].ravel().astype(np.uint8)
      print("Done.")
      return return_X, return_y
    else:
      return X, y


def discretize_colum(data_clm, num_values=10):
    """ Discretize a column by quantiles """
    r = np.argsort(data_clm)
    bin_sz = (len(r) / num_values) + 1  # make sure all quantiles are in range 0-(num_quarts-1)
    q = r // bin_sz
    return q


def load_data(dataset="covertype"):

    print("Loading dataset " + dataset + "...")

    if dataset == "breast_cancer":
      loaded_dataset = load_breast_cancer()
      X, y = select_NROWS_NCOLS(loaded_dataset.data, loaded_dataset.target)

    elif dataset == "iris":
      loaded_dataset = load_iris()
      X, y = loaded_dataset.data, loaded_dataset.target

    elif dataset == "wine":
      loaded_dataset = load_wine()
      X, y = loaded_dataset.data, loaded_dataset.target

    elif dataset == "covertype":  # Multi-class classification dataset
        X_temp, y_temp = sklearn.datasets.fetch_covtype(return_X_y=True)
        print("Is data frame: ", isinstance(X_temp, pd.DataFrame))
        X, y = select_NROWS_NCOLS(X_temp, y_temp)

    elif dataset == "kddcup99":  # Multi-class classification dataset with categorical data
        X_temp, y_temp = sklearn.datasets.fetch_kddcup99(return_X_y=True)
        print("Is data frame: ", isinstance(X_temp, pd.DataFrame))
        X, y = select_NROWS_NCOLS(X_temp, y_temp)

    elif dataset == "make":
        X, y = make_classification(n_samples=1000,
                                   n_features=10,
                                   random_state=123)

    elif dataset == "adult" or dataset == "adultcat":  # Binary classification dataset with categorical data, if you pass AdultCat, the numerical columns will be discretized.
        url_data = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"

        features = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital-status', 'occupation',
                    'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']
        label = "income"
        columns = features + [label]
        df = pd.read_csv(url_data, names=columns)

        # Fill NaN with something better?
        df.fillna(0, inplace=True)
        if dataset == "adultcat":
            columns_to_discr = [('age', 10), ('fnlwgt', 25), ('capital-gain', 10), ('capital-loss', 10),
                                ('hours-per-week', 10)]
            for clm, nvals in columns_to_discr:
                df[clm] = discretize_colum(df[clm], num_values=nvals)
                df[clm] = df[clm].astype(int).astype(str)
            df['education_num'] = df['education_num'].astype(int).astype(str)
        X_temp = df[features].to_numpy()
        y_temp = df[label].to_numpy()
        return select_NROWS_NCOLS(X_temp, y_temp)
    else:
        raise AttributeError("Dataset \"" + dataset + "\" not available")

    print("Dataset loaded!")
    print(f"X.shape: {X.shape}")
    print(f"y.shape: {y.shape}")

    return X, y


# 2 - cv functions

In [6]:
# Define a custom scoring function using accuracy_score with normalize=True
def balanced_accuracy(y_true, y_pred):
    return balanced_accuracy_score(y_true, y_pred)

balanced_accuracy_scorer = make_scorer(balanced_accuracy)

## 2 - 0 Random Forest & Extra Trees & TabPFN

In [7]:
def param_free_cv(X_train, X_test,
                  y_train, y_test,
                  method="rf"):
  if method == "rf":
    clf = RandomForestClassifier()
  elif method == "et":
    clf = ExtraTreesClassifier()
  elif method == "tabpfn":
    clf = TabPFNClassifier(device='cpu',
                           N_ensemble_configurations=32)
  cv_in = -cross_val_score(clf,
                          X_train, y_train,
                          scoring=balanced_accuracy_scorer,
                          cv=5, verbose=0).mean()
  y_pred = clf.fit(X_train, y_train).predict(X_test)
  err_out = balanced_accuracy_score(y_test, y_pred)
  return -cv_in, err_out

## 2 - 1 BCN

In [8]:
def bcn_cv(X_train, y_train,
               B = 10, nu = 0.335855,
               lam = 10**0.7837525,
               r = 1 - 10**(-5.470031),
               tol = 10**-7,
           ):

  estimator  = bcn.BCNClassifier(B = int(B),
                                 nu = nu,
                                 lam = lam,
                                 r = r,
                                 tol = tol,
                                 activation="tanh",
                                 type_optim="nlminb",
                                 show_progress = False)

  return -cross_val_score(estimator, X_train, y_train,
                          scoring=balanced_accuracy_scorer,
                          cv=5, verbose=0).mean()

def optimize_bcn(X_train, X_test, y_train, y_test):
  # objective function for hyperparams tuning
  def crossval_objective(x):
    return bcn_cv(X_train=X_train,
                  y_train=y_train,
                  B = int(x[0]),
                  nu = 10**x[1],
                  lam = 10**x[2],
                  r = 1 - 10**x[3],
                  tol = 10**x[4]
                  )
  gp_opt = gp.GPOpt(objective_func=crossval_objective,
                    lower_bound = np.array([   3,    -6, -10, -10,   -6]),
                    upper_bound = np.array([ 100,  -0.1,  10,  -1, -0.1]),
                    params_names=["B", "nu", "lam", "r", "tol"],
                      n_init=10, n_iter=50, seed=123)
  res = gp_opt.optimize(verbose=1)
  res.best_params["B"] = int(res.best_params["B"])
  res.best_params["nu"] = 10**res.best_params["nu"]
  res.best_params["lam"] = 10**res.best_params["lam"]
  res.best_params["r"] = 1 - 10**res.best_params["r"]
  res.best_params["tol"] = 10**res.best_params["tol"]
  clf = bcn.BCNClassifier(**res.best_params,
                          activation="tanh",
                          type_optim="nlminb",
                          show_progress = False)
  y_pred = clf.fit(X_train, y_train).predict(X_test)
  err_out = balanced_accuracy_score(y_test, y_pred)
  return -res.best_score, err_out

## 2 - 2 xgboost

In [9]:
def xgboost_cv(X_train, y_train,
               n_estimators=100,
               eta=0.1,
               max_depth=3,
               subsample=1.0,
               colsample_bytree=1.0,
               min_child_weight=1,
               seed=123):

  estimator = xgb.XGBClassifier(tree_method="exact",
                                n_estimators=int(n_estimators),
                                eta=eta,
                                max_depth=max_depth,
                                subsample=subsample,
                                colsample_bytree=colsample_bytree,
                                min_child_weight=min_child_weight,
                                random_state=seed, verbosity=0)

  return -cross_val_score(estimator, X_train, y_train,
                          scoring=balanced_accuracy_scorer,
                          cv=5, verbose=0).mean()

def optimize_xgboost(X_train, X_test, y_train, y_test):

  # objective function for hyperparams tuning
  def crossval_objective(x):

    return xgboost_cv(
      X_train=X_train,
      y_train=y_train,
      n_estimators=int(x[0]),
      eta=x[1],
      max_depth=int(x[2]),
      subsample=x[3],
      colsample_bytree=x[4],
      min_child_weight=int(x[5]))

  gp_opt = gp.GPOpt(objective_func=crossval_objective,
                    lower_bound = np.array([ 10, 0.001, 1, 0.5, 0.5, 1]),
                    upper_bound = np.array([250,   0.4, 10, 1.0, 1.0, 20]),
                    params_names=["n_estimators", "eta", "max_depth",
                                  "subsample", "colsample_bytree",
                                  "min_child_weight"],
                      n_init=10, n_iter=50, seed=123)
  res = gp_opt.optimize(verbose=1)
  res.best_params["n_estimators"] = int(res.best_params["n_estimators"])
  res.best_params["max_depth"] = int(res.best_params["max_depth"])
  res.best_params["min_child_weight"] = int(res.best_params["min_child_weight"])
  clf = xgb.XGBClassifier(tree_method="exact",
                          random_state=123,
                          verbosity=0,
                          **res.best_params)
  y_pred = clf.fit(X_train, y_train).predict(X_test)
  err_out = balanced_accuracy_score(y_test, y_pred)
  return -res.best_score, err_out


# 3 - CV

In [10]:
results = {"rf": [], "et": [],
       "tabpfn": [],
       "xgboost": [],
       "bcn": []}

In [11]:
from sklearn.exceptions import ConvergenceWarning

with warnings.catch_warnings():

    warnings.simplefilter("ignore")
    warnings.filterwarnings("ignore", category=Warning)
    warnings.filterwarnings("ignore", category=ConvergenceWarning, module="sklearn")
    warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")

    for dataset in DATASETS[:4]:

      X, y = load_data(dataset)

      X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                          test_size=0.3,
                                                          random_state=3137)
      #try:
      print("\n rf --------------------")
      temp = param_free_cv(X_train, X_test, y_train, y_test, method="rf")
      print(temp)
      results["rf"].append(temp)
      print("\n et --------------------")
      temp = param_free_cv(X_train, X_test, y_train, y_test, method="et")
      print(temp)
      results["et"].append(temp)
      print("\n tabpfn --------------------")
      temp = param_free_cv(X_train, X_test, y_train, y_test, method="tabpfn")
      print(temp)
      results["tabpfn"].append(temp)
      print("\n xgboost --------------------")
      temp = optimize_xgboost(X_train, X_test, y_train, y_test)
      print(temp)
      results["xgboost"].append(temp)
      print("\n bcn --------------------")
      temp = optimize_bcn(X_train, X_test, y_train, y_test)
      print(temp)
      results["bcn"].append(temp)
      #except:
      #  print(f"\n MISSED: {dataset} --------------------")
      #  continue


Loading dataset iris...
Dataset loaded!
X.shape: (150, 4)
y.shape: (150,)

 rf --------------------
(0.936904761904762, 0.9696969696969697)

 et --------------------
(0.936904761904762, 0.9696969696969697)

 tabpfn --------------------


<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen impo

(0.9464285714285715, 0.9696969696969697)

 xgboost --------------------

 Creating initial design... 

10/10 [██████████████████████████████] - 5s 454ms/step

 ...Done. 


 Optimization loop... 

50/50 [██████████████████████████████] - 39s 776ms/step
(0.9464285714285715, 0.9696969696969697)

 bcn --------------------

 Creating initial design... 

10/10 [██████████████████████████████] - 9s 862ms/step

 ...Done. 


 Optimization loop... 

 1/50 [..............................] - ETA: 8s

/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


 2/50 [█.............................] - ETA: 8s

...

10/50 [██████........................] - ETA: 2:43

/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


11/50 [██████........................] - ETA: 2:26

/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


50/50 [██████████████████████████████] - 283s 6s/step
(0.9697870939420545, 0.9803571428571429)


In [12]:
from google.colab import files

joblib.dump(results, "results.pkl")

files.download('results.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
scores_array = np.empty((4, 5))

scores_array[:,:] = np.nan

scores_df = pd.DataFrame(scores_array, columns=["rf",
                                                "et", "tabpfn",
                                                "xgboost", "bcn"])

In [21]:
for idx, (key, value) in enumerate(results.items()):
  print(f"{key}: {value}")
  for idx2, elt in enumerate(value):
    scores_df[key][idx2]=elt[1]

rf: [(0.936904761904762, 0.9696969696969697), (0.9783333333333333, 1.0), (0.9531168743359413, 0.9366076527698458), (0.9859447892153522, 1.0)]
et: [(0.936904761904762, 0.9696969696969697), (0.9866666666666667, 0.9824561403508771), (0.9457470298464214, 0.9391062250142777), (0.9914813188957176, 1.0)]
tabpfn: [(0.9464285714285715, 0.9696969696969697), (0.9933333333333334, 0.9777777777777779), (0.9550776586496668, 0.9633780696744718), (0.9842698273339187, 1.0)]
xgboost: [(0.9464285714285715, 0.9696969696969697), (0.9866666666666667, 0.9657894736842105), (0.9600134260600791, 0.9391062250142777), (0.9858630898689471, 1.0)]
bcn: [(0.9535714285714286, 0.9696969696969697), (0.9933333333333334, 1.0), (0.9535901671013232, 0.9755853797829812), (0.9697870939420545, 0.9803571428571429)]


In [23]:
scores_df

,rf,et,tabpfn,xgboost,bcn
0,0.9697,0.9697,0.9697,0.9697,0.9697
1,1.0000,0.9825,0.9778,0.9658,1.0000
2,0.9366,0.9391,0.9634,0.9391,0.9756
3,1.0000,1.0000,1.0000,1.0000,0.9804


In [22]:
scores_ranks_df = scores_df.rank(axis=1, method='max', ascending=False)
display(scores_ranks_df)
scores_ranks_df.describe()

,rf,et,tabpfn,xgboost,bcn
0,5.0000,5.0000,5.0000,5.0000,5.0000
1,2.0000,3.0000,4.0000,5.0000,2.0000
2,5.0000,4.0000,2.0000,4.0000,1.0000
3,4.0000,4.0000,4.0000,4.0000,5.0000


,rf,et,tabpfn,xgboost,bcn
count,4.0000,4.0000,4.0000,4.0000,4.0000
mean,4.0000,4.0000,3.7500,4.5000,3.2500
std,1.4142,0.8165,1.2583,0.5774,2.0616
min,2.0000,3.0000,2.0000,4.0000,1.0000
25%,3.5000,3.7500,3.5000,4.0000,1.7500
50%,4.5000,4.0000,4.0000,4.5000,3.5000
75%,5.0000,4.2500,4.2500,5.0000,5.0000
max,5.0000,5.0000,5.0000,5.0000,5.0000
